In [1]:
from icalendar import Calendar, Event
from faker import Faker
fake = Faker()
import pandas as pd
from pathlib import Path
import os.path
from os import path
import sys
from calendar import monthrange
from datetime import datetime, timedelta
import uuid
import random

In [2]:
##### User-definable options, start editing here #####

break_length_options = [0,30,60,90]
break_length_weights = [0.25,0.5,0.2,0.05]
event_length_options = [30,60,90,120,240]
event_length_weights = [0.6,0.2,0.1,0.05,0.01]

user_input_year = 2023

user_input_month = 11

EmployeeCalendars = pd.DataFrame(columns = ['EmployeeID', 'EmployeeName', 'EmployeeTeam', 'SelectedTimeZone', 'EmployeeDay', 'UserBeginTime', 'UserEndTime'])
days_of_week = ['0', '1', '2', '3', '4']

print(EmployeeCalendars)

Empty DataFrame
Columns: [EmployeeID, EmployeeName, EmployeeTeam, SelectedTimeZone, EmployeeDay, UserBeginTime, UserEndTime]
Index: []


In [3]:
num_people = int(input("Enter the number of people: "))
num_teams = int(input("Enter the number of teams: "))
Teams = ["Marketing", "Finance", "Human Resources", "Sales", "IT (Information Technology)", "Operations", "Research and Development", "Customer Support", "Legal", "Product Development"]
selected_teams = random.sample(Teams, num_teams)
for i in range(num_people):
    EmployeeID = i
    EmployeeName = fake.name()
    SelectedTimeZone = random.choice(["San Francisco", "New York City", "London"])
    EmployeeTeam = random.choice(selected_teams) 
    for day in days_of_week:
        EmployeeDay = day
        if SelectedTimeZone == "San Francisco":
            random_hour = random.randint(11, 13) 
        elif SelectedTimeZone == "New York City":
            random_hour = random.randint(8, 10) 
        elif SelectedTimeZone == "London":
            random_hour = random.randint(2, 4) 
        random_minute = random.choice([0, 30])
        current_time = datetime.now()
        UserBeginTime = current_time.replace(hour = random_hour, minute = random_minute)
        UserBeginTime = UserBeginTime - timedelta(days = current_time.weekday()) + timedelta(days = int(EmployeeDay))
        UserEndTime = UserBeginTime + timedelta(hours=8)

        new_row = pd.DataFrame({
            'EmployeeID': [EmployeeID],
            'EmployeeName': [EmployeeName],
            'EmployeeTeam': [EmployeeTeam],
            'SelectedTimeZone': [SelectedTimeZone],
            'EmployeeDay': [EmployeeDay],
            'UserBeginTime': [UserBeginTime],
            'UserEndTime': [UserEndTime]
        })
        EmployeeCalendars = pd.concat([EmployeeCalendars, new_row], ignore_index=True)

print(EmployeeCalendars)

Enter the number of people:  3
Enter the number of teams:  2


   EmployeeID      EmployeeName                 EmployeeTeam SelectedTimeZone  \
0           0     William Stone  IT (Information Technology)    New York City   
1           0     William Stone  IT (Information Technology)    New York City   
2           0     William Stone  IT (Information Technology)    New York City   
3           0     William Stone  IT (Information Technology)    New York City   
4           0     William Stone  IT (Information Technology)    New York City   
5           1  Jeffrey Mckinney  IT (Information Technology)           London   
6           1  Jeffrey Mckinney  IT (Information Technology)           London   
7           1  Jeffrey Mckinney  IT (Information Technology)           London   
8           1  Jeffrey Mckinney  IT (Information Technology)           London   
9           1  Jeffrey Mckinney  IT (Information Technology)           London   
10          2    Billy Davidson  IT (Information Technology)    San Francisco   
11          2    Billy David

C:\Users\terro\AppData\Local\Temp\ipykernel_24464\1270753844.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  EmployeeCalendars = pd.concat([EmployeeCalendars, new_row], ignore_index=True)


In [4]:
calendar_folder = str(Path.home()) + "\\Python"
os.chmod(calendar_folder, 0o777)
files = os.listdir(calendar_folder)
for f in files:
    os.remove(calendar_folder + "\\" + f)

In [5]:
# https://stackoverflow.com/questions/3408097/parsing-files-ics-icalendar-using-python
for index, row in EmployeeCalendars.iterrows():
    
    if (EmployeeCalendars.EmployeeDay[index] == "0"):
        cal = Calendar()
        cal.add('prodid', 'PYGen')
        cal.add('version', '2.0')
    
    event_end = row['UserBeginTime']
    day_end = row['UserEndTime']

    # Pick if there's a break or an event, either way add time to it and iterate 
    while (event_end < day_end):
        event = Event()
        event.add('summary', 'Python Meeting Randomly Generated')
        event.add('dtstamp', datetime.now())
        break_length = timedelta(minutes=random.choices(break_length_options, weights=break_length_weights, k=1)[0])
        event_end = event_end + break_length
        event_begin = event_end
        event.add('dtstart', event_begin)
        event_length = timedelta(minutes=random.choices(event_length_options, weights=event_length_weights, k=1)[0])
        event_end = event_begin + event_length
        event.add('dtend', event_end)
        event.add('uid', str(EmployeeID) + EmployeeName.replace(" ", ""))
        event.add('priority', 5)

        if (event_end < day_end):
            cal.add_component(event)

    # Open the file with the unique filename and write the calendar data
    if (EmployeeCalendars.EmployeeDay[index] == "4"):
        filename = os.path.join(calendar_folder, str(EmployeeCalendars.EmployeeID[index]) + EmployeeCalendars.EmployeeName[index].replace(" ", "") + ".ics")
        with open(filename, 'wb') as f:
            f.write(cal.to_ical())
        print("\033[92miCalendar files generated successfully and have been saved to: " + calendar_folder + "\033[0m")


iCalendar files generated successfully and have been saved to: C:\Users\terro\Python
iCalendar files generated successfully and have been saved to: C:\Users\terro\Python
iCalendar files generated successfully and have been saved to: C:\Users\terro\Python
